In [15]:
import numpy as np
import pandas as pd
import xarray as xr

In [16]:
def create_dummy_input_file(filename, F):
    """Create an input file for testing. Does not use unlimited
    dimensions, creates one time record only."""

    import netCDF4

    nc = netCDF4.Dataset(filename, "w")

    Mx = 88
    My = 152
    Mz = 11
    for name, length in [["x", Mx], ["y", My], ["z", Mz], ["time", 1]]:
        nc.createDimension(name, length)
        nc.createVariable(name, "f8", (name,))

    # use X and Y ranges corresponding to a grid covering Greenland
    x = np.linspace(-669650.0, 896350.0, Mx)
    y = np.linspace(-3362600.0, -644600.0, My)
    z = np.linspace(0, 4000.0, Mz)
    # the single time record
    time = [0.0]

    for name, data in [["x", x], ["y", y], ["z", z], ["time", time]]:
        nc.variables[name][:] = data

    nc.proj4 = "epsg:3413"

    xx, yy = np.meshgrid(x, y)

    def write(prefix, dimensions):
        "Write test data to the file using given storage order."
        name = prefix + "_".join(dimensions)

        slices = {"x": slice(0, Mx), "y": slice(0, My), "time": 0, "z": None}

        if "z" in dimensions:
            # set fill_value and coordinates in variables with the z
            # dimensions and not others (just so that we can get
            # better test coverage)
            variable = nc.createVariable(name, "f8", dimensions, fill_value=-2e9)
            variable.coordinates = "lon lat"
        else:
            variable = nc.createVariable(name, "f8", dimensions)

        variable.long_name = name + " (let's make it long!)"

        # set indexes for all dimensions (z index will be re-set below)
        indexes = [Ellipsis] * len(dimensions)
        for k, d in enumerate(dimensions):
            indexes[k] = slices[d]

        # transpose 2D array if needed
        if dimensions.index("y") < dimensions.index("x"):

            def T(x):
                return x

        else:
            T = np.transpose

        if "z" in dimensions:
            for k in range(Mz):
                indexes[dimensions.index("z")] = k
                variable[indexes] = T(F(xx, yy, z[k]))
        else:
            variable[indexes] = T(F(xx, yy, 0))

    from itertools import permutations

    def P(x):
        return list(permutations(x))

    for d in sorted(P(["x", "y"]) + P(["time", "x", "y"])):
        write("test_2D_", d)

    for d in sorted(P(["x", "y", "z"]) + P(["time", "x", "y", "z"])):
        write("test_3D_", d)

    nc.close()


In [17]:
    def F(x, y, z):
        """A function linear in x, y, and z. Used to test our interpolation
        scheme."""
        return 10.0 + 0.01 * x + 0.02 * y + 0.03 + 0.04 * z

    # create a test file
    import tempfile
    import os

    fd, filename = tempfile.mkstemp(suffix=".nc", prefix="extract_profile_test_")
    os.close(fd)

    create_dummy_input_file(filename, F)
    create_dummy_input_file("test.nc", F)


In [199]:
def create_dummy_input_file_xr(filename, F):
    """Create an input file for testing. Does not use unlimited
    dimensions, creates one time record only."""

    Mx = 88
    My = 152
    Mz = 11
    Mt = 1
    
    # use X and Y ranges corresponding to a grid covering Greenland
    x = np.linspace(-669650.0, 896350.0, Mx)
    y = np.linspace(-3362600.0, -644600.0, My)
    z = np.linspace(0, 4000.0, Mz)

    xx, yy = np.meshgrid(x, y)

    def write(prefix, dimensions):
        "Write test data to the file using given storage order."

        slices = {"x": slice(0, Mx), "y": slice(0, My), "time": 0, "z": None}
        dim_map = {"x": Mx, "y": My, "z": Mz, "time": Mt}

        # set indexes for all dimensions (z index will be re-set below)
        indexes = [Ellipsis] * len(dimensions)
        for k, d in enumerate(dimensions):
            indexes[k] = slices[d]

        # transpose 2D array if needed
        if dimensions.index("y") < dimensions.index("x"):

            def T(x):
                return x

        else:
            T = np.transpose

        dims = [dim_map[d] for d in dimensions]
        variable = np.zeros(dims)
        if "z" in dimensions:
            for k in range(Mz):
                indexes[dimensions.index("z")] = k
                variable[*indexes] = T(F(xx, yy, z[k]))
        else:
            variable[*indexes] = T(F(xx, yy, 0))
            
        return (dimensions, variable, {"long_name": name + " (make it long!)"})

    from itertools import permutations

    def P(x):
        return list(permutations(x))

    data_vars = dict()
    for d in sorted(P(["x", "y"]) + P(["time", "x", "y"])):
        prefix = "test_2D_"
        name = prefix + "_".join(d)
        data_vars[name] = write(prefix, d)

    for d in sorted(P(["x", "y", "z"]) + P(["time", "x", "y", "z"])):
        prefix = "test_3D_"
        name = prefix + "_".join(d)
        data_vars[name] = write(prefix, d)
        
    ds = xr.Dataset(
        data_vars=data_vars,
        coords=dict(
            time=(["time"], [0], {}),
            z=(["z"], z, {"_FillValue": False, "units": "m"}),
            y=(["y"], y, {"_FillValue": False, "units": "m", "axis": "Y", "standard_name": "projection_y_coordinate"}),
            x=(["x"], x, {"_FillValue": False, "units": "m", "axis": "X", "standard_name": "projection_x_coordinate"}),
        ),
        attrs=dict(description="Test data."),
    )
    ds.to_netcdf(filename)
    return ds



In [200]:
create_dummy_input_file_xr("test_xr.nc", F)

<xarray.Dataset>
Dimensions:             (time: 1, x: 88, y: 152, z: 11)
Coordinates:
  * time                (time) int64 0
  * z                   (z) float64 0.0 400.0 800.0 ... 3.2e+03 3.6e+03 4e+03
  * y                   (y) float64 -3.363e+06 -3.345e+06 ... -6.446e+05
  * x                   (x) float64 -6.696e+05 -6.516e+05 ... 8.964e+05
Data variables: (12/38)
    test_2D_time_x_y    (time, x, y) float64 -7.394e+04 ... -3.918e+03
    test_2D_time_y_x    (time, y, x) float64 -7.394e+04 ... -3.918e+03
    test_2D_x_time_y    (x, time, y) float64 -7.394e+04 ... -3.918e+03
    test_2D_x_y         (x, y) float64 -7.394e+04 -7.358e+04 ... -3.918e+03
    test_2D_x_y_time    (x, y, time) float64 -7.394e+04 ... -3.918e+03
    test_2D_y_time_x    (y, time, x) float64 -7.394e+04 ... -3.918e+03
    ...                  ...
    test_3D_z_x_time_y  (z, x, time, y) float64 -7.394e+04 ... -3.758e+03
    test_3D_z_x_y       (z, x, y) float64 -7.394e+04 -7.358e+04 ... -3.758e+03
    test_3D_z_x_y_time  (z, x, y, time) float64 -7.394e+04 ... -3.758e+03
    test_3D_z_y_time_x  (z, y, time, x) float64 -7.394e+04 ... -3.758e+03
    test_3D_z_y_x       (z, y, x) float64 -7.394e+04 -7.376e+04 ... -3.758e+03
    test_3D_z_y_x_time  (z, y, x, time) float64 -7.394e+04 ... -3.758e+03
Attributes:
    description:  Test data.

In [85]:

Mx = 88
My = 152
Mz = 11

slices = {"x": slice(0, Mx), "y": slice(0, My), "time": 0, "z": None}

In [54]:
a =slices["y"]

In [48]:
np.zeros(slices)

TypeError: expected a sequence of integers or a single integer, got '{'x': slice(0, 88, None), 'y': slice(0, 152, None), 'time': 0, 'z': None}'

In [67]:
dimension = ('time', 'x', 'y')
dim_map = {"x": Mx, "y": My, "z": None, "time": 1}


In [57]:
slices

{'x': slice(0, 88, None), 'y': slice(0, 152, None), 'time': 0, 'z': None}

In [58]:
d[0]

'time'

In [66]:
[*(dims[d])]

KeyError: ('time', 'x', 'y')

In [71]:
np.zeros([dim_map[d] for d in dimension]).shape

(1, 88, 152)

In [80]:
variable = np.zeros([dim_map[d] for d in dimension])


In [82]:
variable[0, slice(0, 88, None), slice(0, 152, None)]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [91]:
ds

NameError: name 'ds' is not defined

In [95]:
    """
    Return a dataset with missing values
    """

    start_date = "1980-01-01"
    calendar = "standard"
    periodicity = "YS"
    reference_time = pd.Timestamp("1980-01-01")

    nx = 101
    ny = 101
    nt = 2

    dates = xr.cftime_range(start_date, periods=nt, freq=periodicity, calendar=calendar)
    x = np.linspace(-100_000, 100_000, nx)
    y = np.linspace(-100_000, 100_000, ny)
    t = np.arange(nt)
    X, Y = np.meshgrid(x, y)

    t_mask = ((np.abs(X)<25_000) & (np.abs(Y)<25_000)).reshape(1, ny, nx)
    t_mask = np.repeat(t_mask, repeats=nt, axis=0)
    temperature = np.exp(((X/100_000)**2 + (Y/100_000)**2)).reshape(1, ny, nx)
    temperature = np.repeat(temperature, repeats=nt, axis=0)

    p_mask = ((np.abs(X)<25_000) & (Y<-25_000)).reshape(1, ny, nx)
    p_mask = np.repeat(p_mask, repeats=nt, axis=0)
    precipitation = (np.exp(((X/100_000)**2 + (Y/100_000)**2)) ** 2).reshape(1, ny, nx)
    precipitation = np.repeat(precipitation, repeats=nt, axis=0)

    ds_true = xr.Dataset(
        data_vars=dict(
            temperature=(["time", "x", "y"], temperature, {"units": "Celsius"}),
            precipitation=(["time", "y", "x"], precipitation, {"units": "mm/day"}),
        ),
        coords=dict(
            time=dates,
            y=(["y"], y, {"_FillValue": False, "units": "m", "axis": "Y", "standard_name": "projection_y_coordinate"}),
            x=(["x"], x, {"_FillValue": False, "units": "m", "axis": "X", "standard_name": "projection_x_coordinate"}),
            reference_time=reference_time,
        ),
        attrs=dict(description="Test data."),
    )


In [96]:
ds_true

<xarray.Dataset>
Dimensions:         (time: 2, x: 101, y: 101)
Coordinates:
  * time            (time) object 1980-01-01 00:00:00 1981-01-01 00:00:00
  * y               (y) float64 -1e+05 -9.8e+04 -9.6e+04 ... 9.8e+04 1e+05
  * x               (x) float64 -1e+05 -9.8e+04 -9.6e+04 ... 9.8e+04 1e+05
    reference_time  datetime64[ns] 1980-01-01
Data variables:
    temperature     (time, x, y) float64 7.389 7.102 6.832 ... 6.832 7.102 7.389
    precipitation   (time, y, x) float64 54.6 50.44 46.67 ... 46.67 50.44 54.6
Attributes:
    description:  Test data.